In [ ]:
!pip install syft==0.2.4

In [ ]:
%%time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import logging
import argparse
import sys
import pandas as pd

import syft as sy
from syft.workers.websocket_client import WebsocketClientWorker
from syft.frameworks.torch.fl import utils
from syft.frameworks.torch.nn import LSTM
from syft.federated.floptimizer import Optims

import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, callbacks
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

logger = logging.getLogger(__name__)


class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 50
        self.federate_after_n_batches = 100
        self.lr = 0.001
        self.cuda = False
        self.seed = 1


args = Arguments()

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.word_embeddings = nn.Embedding(39, 128)
        self.lstm = LSTM(128, 128, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.linear1 = nn.Linear(47*128, 100)
        self.linear2 = nn.Linear(100, 1)


    def init_hidden(self):
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))


    def forward(self, x):
        embedded = self.word_embeddings(x)
        x, (ht, ct) = self.lstm(embedded)
        x = x.contiguous().view(-1, 47*128)
        x = F.relu(x)
        x = self.dropout(x)
        x = F.relu(self.linear1(x))
        x = F.sigmoid(self.linear2(x))
        return x


def test_worker(model, device, test_loader, worker):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.binary_cross_entropy(output.squeeze(), target.float(), reduction="sum").item()  # sum up batch loss
            pred = torch.round(output.squeeze())
            correct += pred.eq(target.float().view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    logger.debug("\n")
    accuracy = 100.0 * correct / len(test_loader.dataset)
    logger.info(
        "Test on {}'s dataset: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            worker.id, test_loss, correct, len(test_loader.dataset), accuracy
        )
    )

def train_on_batches(worker, batches, model_in, device, lr):
    model = model_in.copy()
    optimizer = optim.Adam(model.parameters(), lr = 0.001)
    model.train()
    model.send(worker)
    loss_local = False

    for batch_idx, (data, target) in enumerate(batches):
        loss_local = False
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.binary_cross_entropy(output.squeeze(), target.float())
        loss.backward()
        optimizer.step()
        if batch_idx % int(len(batches)/2) == 0:
            loss = loss.get()
            loss_local = True
            logger.debug(
                "Train Worker {}: ({:.0f}%)]\tLoss: {:.6f}".format(
                    worker.id,
                    100.0 * batch_idx / len(batches),
                    loss.item(),
                )
            )

    if not loss_local:
        loss = loss.get()
    model.get()
    return model, loss


def get_next_batches(fdataloader: sy.FederatedDataLoader, nr_batches: int):
    batches = {}
    for worker_id in fdataloader.workers:
        worker = fdataloader.federated_dataset.datasets[worker_id].location
        batches[worker] = []
    try:
        for i in range(nr_batches):
            next_batches = next(fdataloader)
            for worker in next_batches:
                batches[worker].append(next_batches[worker])
    except StopIteration:
        pass
    return batches


def train(
    model, device, federated_train_loader, lr, federate_after_n_batches, epoch, test_loader, abort_after_one=False
):
    model.train()

    nr_batches = federate_after_n_batches

    models = {}
    loss_values = {}

    iter(federated_train_loader)  # initialize iterators
    batches = get_next_batches(federated_train_loader, nr_batches)
    counter = 0

    while True:
        if counter==0:
            logger.debug(f"Starting training epoch {epoch}")
        data_for_all_workers = True
        load = []
        for worker in batches:
            if worker not in load:
               load.append(worker)
            curr_batches = batches[worker]
            if curr_batches:
                models[worker], loss_values[worker] = train_on_batches(
                    worker, curr_batches, model, device, lr
                )
            else:
                data_for_all_workers = False

        counter += nr_batches
        if not data_for_all_workers:
            for worker in load:
                test_worker(models[worker], device, test_loader, worker)
            break

        if counter % 200==0:
            model = utils.federated_avg(models)
            logger.debug(f"Federated-averaging after trained on given batches")

        batches = get_next_batches(federated_train_loader, nr_batches)
        if abort_after_one:
            break
    return model


def test(args, model, device, test_loader, max_accuracy, epochs_no_improve):
    model.eval()
    test_loss = 0
    correct = 0
    pred_list = []
    test_list = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.binary_cross_entropy(output.squeeze(), target.float(), reduction='sum').item() # sum up batch loss
            pred = torch.round(output.squeeze())
            correct += pred.eq(target.float().view_as(pred)).sum().item()
            pred_list.append(pred.numpy())
            test_list.append(target.float().view_as(pred).numpy())

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    accuracy = 100. * correct / len(test_loader.dataset)

    if accuracy > max_accuracy:
        epochs_no_improve = 0
        max_accuracy = accuracy
    else:
        epochs_no_improve += 1

    return pred_list, test_list, max_accuracy, epochs_no_improve, accuracy


DATA_FILE = '/content/drive/My Drive/Colab Notebooks/MIXED_CORRECT/traindata_augmented_sampled.pkl'


KEEP_LABELS_DICT = {
    "alice": ['ramnit', 'kraken', 'simda', 'nymaim2', 'pizd'],
    "bob": ['banjori', 'pykspa', 'ramdo', 'qakbot', 'cryptolocker', 'gozi'],
    "charlie": ['locky', 'corebot', 'dircrypt', 'matsnu', 'suppobox'],
    "testing": ['pykspa', 'simda', 'kraken', 'qakbot', 'cryptolocker', 'locky', 'ramdo', 'ramnit', 'banjori', 'corebot', 'dircrypt', 'gozi', 'nymaim2', 'pizd', 'matsnu', 'suppobox']
}

original = ['benign']

dga = ['pykspa', 'simda', 'kraken', 'qakbot', 'cryptolocker', 'locky', 'ramdo', 'ramnit', 'banjori', 'corebot', 'dircrypt', 'gozi', 'nymaim2', 'pizd', 'matsnu', 'suppobox']



def init_data(id):
    data = pickle.load(open(DATA_FILE, 'rb'))
    test_dataset=data[:64000]
    data = data[64000:]
    X = data["text"]
    labels = data["label"]
    data_client = data.loc[data['label'].isin(KEEP_LABELS_DICT[id])]
    data_benign = data.loc[data['label'].isin(original)]
    if id=='alice':
        data_client = pd.concat([data_client, data_benign[:45000]], ignore_index=True, sort=False)
    elif id=='bob':
        data_client = pd.concat([data_client, data_benign[45000:83000]], ignore_index=True, sort=False)
    elif id=='charlie':
        data_client = pd.concat([data_client, data_benign[83000:]], ignore_index=True, sort=False)
    else:
        data_client=test_dataset

    data_client = data_client.sample(frac=1)

    y = [0 if x == 'benign' else 1 for x in data_client['label']]
    valid_chars = {'7': 1, 'i': 2, 'x': 3, '5': 4, 'w': 5, 't': 6, 'v': 7, 'g': 8, 'k': 9, 'd': 10, 'z': 11, '6': 12, '-': 13, '_': 14, 'a': 15, 'p': 16, 'e': 17, '9': 18, 'b': 19, 'f': 20, 'y': 21, 
                   '2': 22, 'c': 23, 'l': 24, 's': 25, 'n': 26, 'h': 27, '3': 28, 'u': 29, 'm': 30, '0': 31, 'r': 32, 'j': 33, '8': 34, 'o': 35, '4': 36, '1': 37, 'q': 38}
    max_features = len(valid_chars) + 1
    max_len = 47
    X = data_client["text"]
    X = [[valid_chars[y] for y in x] for x in X]
    X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_len, padding='pre')

    return np.array(X), np.array(y).reshape(len(y),1), max_features, max_len


def main():
    hook = sy.TorchHook(torch)
    bob = sy.VirtualWorker(hook, id="bob")  # define remote worker bob
    alice = sy.VirtualWorker(hook, id="alice")  # and alice
    charlie = sy.VirtualWorker(hook, id="charlie") #and bob

    workers = [alice, bob, charlie]

    use_cuda = args.cuda and torch.cuda.is_available()

    torch.manual_seed(args.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    for i in ['alice', 'bob', 'charlie', 'testing']:
        X, Y, max_features, max_len = init_data(i)
        if i=='testing':
            test_data = torch.utils.data.TensorDataset(torch.LongTensor(X), torch.LongTensor(Y))
        else:
            selected_data = torch.LongTensor(X)
            selected_targets = torch.LongTensor(Y).squeeze(1)
            if i=='bob':
                bob_train_dataset = sy.BaseDataset(data=selected_data, targets=selected_targets).send(bob)
            if i=='alice':
                alice_train_dataset = sy.BaseDataset(data=selected_data, targets=selected_targets).send(alice)
            if i=='charlie':
                charlie_train_dataset = sy.BaseDataset(data=selected_data, targets=selected_targets).send(charlie)

    federated_train_dataset = sy.FederatedDataset([bob_train_dataset, alice_train_dataset, charlie_train_dataset])

    federated_train_loader = sy.FederatedDataLoader(federated_train_dataset,
        batch_size=args.batch_size,
        shuffle=True,
        iter_per_worker=True,
        **kwargs,
    )



    test_loader = torch.utils.data.DataLoader(test_data,
        batch_size=args.test_batch_size, shuffle=True, **kwargs)

    model = Net().to(device)


    max_accuracy = 0.0
    epochs_no_improve = 0  
    accuracy_list = [0.0]
    state_dict = torch.load("/content/drive/My Drive/Colab Notebooks/MIXED_CORRECT/MODELS_MIXED/Federated_Mixed/BATCH/LSTM/A/12.pt")
    model.load_state_dict(state_dict)
    for epoch in range(13, args.epochs + 1):
        model = train(model, device, federated_train_loader, args.lr, args.federate_after_n_batches, epoch, test_loader)
        pred_list, test_list, max_accuracy, epochs_no_improve, accuracy = test(args, model, device, test_loader, max_accuracy, epochs_no_improve)
        torch.save(model.state_dict(), "/content/drive/My Drive/Colab Notebooks/MIXED_CORRECT/MODELS_MIXED/Federated_Mixed/BATCH/LSTM/A/"+str(epoch)+".pt")
        accuracy_list.append(accuracy)
        print(accuracy_list)
        if epochs_no_improve > 2:
           print('Early stopping!')
           break


    pred_list = [a.squeeze().tolist() for a in pred_list]
    total = []
    for i in pred_list:
        total += i
    pred_list = total

    test_list = [a.squeeze().tolist() for a in test_list]
    total = []
    for i in test_list:
        total += i
    test_list = total

    pred_list = ['benign' if x==0 else 'dga' for x in pred_list]
    test_list = ['benign' if x==0 else 'dga' for x in test_list]


    from sklearn.metrics import classification_report
    print(classification_report(test_list, pred_list))

    from sklearn.metrics import confusion_matrix
    import seaborn as sns
    confusion_matrix_df = pd.DataFrame(confusion_matrix(test_list, pred_list))
    print(confusion_matrix_df)
    sns.heatmap(confusion_matrix_df, annot=True)

if __name__ == "__main__":
    FORMAT = "%(asctime)s %(levelname)s %(filename)s(l:%(lineno)d) - %(message)s"
    LOG_LEVEL = logging.DEBUG
    logging.basicConfig(format=FORMAT, level=LOG_LEVEL)

    websockets_logger = logging.getLogger("websockets")
    websockets_logger.setLevel(logging.DEBUG)
    websockets_logger.addHandler(logging.StreamHandler())

    main()

2020-09-20 21:18:22,171 DEBUG <timed exec>(l:153) - Starting training epoch 13
2020-09-20 21:18:23,442 DEBUG <timed exec>(l:112) - Train Worker bob: (0%)]	Loss: 0.166174
2020-09-20 21:18:53,826 DEBUG <timed exec>(l:112) - Train Worker bob: (50%)]	Loss: 0.069294
2020-09-20 21:19:23,887 DEBUG <timed exec>(l:112) - Train Worker alice: (0%)]	Loss: 0.074732
2020-09-20 21:19:54,114 DEBUG <timed exec>(l:112) - Train Worker alice: (50%)]	Loss: 0.098570
2020-09-20 21:20:25,043 DEBUG <timed exec>(l:112) - Train Worker charlie: (0%)]	Loss: 0.037184
2020-09-20 21:20:56,254 DEBUG <timed exec>(l:112) - Train Worker charlie: (50%)]	Loss: 0.091601
2020-09-20 21:21:33,643 DEBUG <timed exec>(l:112) - Train Worker bob: (0%)]	Loss: 0.108872
2020-09-20 21:22:05,458 DEBUG <timed exec>(l:112) - Train Worker bob: (50%)]	Loss: 0.046760
2020-09-20 21:22:37,325 DEBUG <timed exec>(l:112) - Train Worker alice: (0%)]	Loss: 0.182109
2020-09-20 21:23:08,347 DEBUG <timed exec>(l:112) - Train Worker alice: (50%)]	Loss:


Test set: Average loss: 0.1515, Accuracy: 60432/64000 (94.42%)

[0.0, 88.696875, 90.4203125, 91.215625, 91.8671875, 92.121875, 92.8203125, 93.2171875, 92.90625, 94.13125, 94.3296875, 94.3140625, 94.478125, 94.425]


2020-09-20 22:03:01,318 DEBUG <timed exec>(l:153) - Starting training epoch 14
2020-09-20 22:03:01,980 DEBUG <timed exec>(l:112) - Train Worker bob: (0%)]	Loss: 0.180791
2020-09-20 22:03:32,615 DEBUG <timed exec>(l:112) - Train Worker bob: (50%)]	Loss: 0.205418
2020-09-20 22:04:03,426 DEBUG <timed exec>(l:112) - Train Worker alice: (0%)]	Loss: 0.165329
2020-09-20 22:04:33,910 DEBUG <timed exec>(l:112) - Train Worker alice: (50%)]	Loss: 0.052740
2020-09-20 22:05:04,284 DEBUG <timed exec>(l:112) - Train Worker charlie: (0%)]	Loss: 0.013172
2020-09-20 22:05:34,690 DEBUG <timed exec>(l:112) - Train Worker charlie: (50%)]	Loss: 0.038260
2020-09-20 22:06:11,016 DEBUG <timed exec>(l:112) - Train Worker bob: (0%)]	Loss: 0.116111
2020-09-20 22:06:41,461 DEBUG <timed exec>(l:112) - Train Worker bob: (50%)]	Loss: 0.150461
2020-09-20 22:07:11,821 DEBUG <timed exec>(l:112) - Train Worker alice: (0%)]	Loss: 0.141449
2020-09-20 22:07:42,109 DEBUG <timed exec>(l:112) - Train Worker alice: (50%)]	Loss:


Test set: Average loss: 0.1532, Accuracy: 60479/64000 (94.50%)

[0.0, 88.696875, 90.4203125, 91.215625, 91.8671875, 92.121875, 92.8203125, 93.2171875, 92.90625, 94.13125, 94.3296875, 94.3140625, 94.478125, 94.425, 94.4984375]


2020-09-20 22:47:20,718 DEBUG <timed exec>(l:153) - Starting training epoch 15
2020-09-20 22:47:21,366 DEBUG <timed exec>(l:112) - Train Worker bob: (0%)]	Loss: 0.154346
2020-09-20 22:47:52,070 DEBUG <timed exec>(l:112) - Train Worker bob: (50%)]	Loss: 0.103425
2020-09-20 22:48:22,407 DEBUG <timed exec>(l:112) - Train Worker alice: (0%)]	Loss: 0.115297
2020-09-20 22:48:52,507 DEBUG <timed exec>(l:112) - Train Worker alice: (50%)]	Loss: 0.119584
2020-09-20 22:49:22,786 DEBUG <timed exec>(l:112) - Train Worker charlie: (0%)]	Loss: 0.088910
2020-09-20 22:49:53,159 DEBUG <timed exec>(l:112) - Train Worker charlie: (50%)]	Loss: 0.030309
2020-09-20 22:50:29,307 DEBUG <timed exec>(l:112) - Train Worker bob: (0%)]	Loss: 0.156566
2020-09-20 22:51:00,369 DEBUG <timed exec>(l:112) - Train Worker bob: (50%)]	Loss: 0.115547
2020-09-20 22:51:30,518 DEBUG <timed exec>(l:112) - Train Worker alice: (0%)]	Loss: 0.158855
2020-09-20 22:52:00,605 DEBUG <timed exec>(l:112) - Train Worker alice: (50%)]	Loss:

...


Test set: Average loss: 0.1373, Accuracy: 60812/64000 (95.02%)

[0.0, 88.696875, 90.4203125, 91.215625, 91.8671875, 92.121875, 92.8203125, 93.2171875, 92.90625, 94.13125, 94.3296875, 94.3140625, 94.478125, 94.425, 94.4984375, 94.8828125, 94.784375, 95.05625, 94.3578125, 95.01875]


2020-09-21 02:37:30,601 DEBUG <timed exec>(l:153) - Starting training epoch 20
2020-09-21 02:37:31,315 DEBUG <timed exec>(l:112) - Train Worker bob: (0%)]	Loss: 0.025770
2020-09-21 02:38:04,918 DEBUG <timed exec>(l:112) - Train Worker bob: (50%)]	Loss: 0.154279
2020-09-21 02:38:38,490 DEBUG <timed exec>(l:112) - Train Worker alice: (0%)]	Loss: 0.180398
2020-09-21 02:39:09,376 DEBUG <timed exec>(l:112) - Train Worker alice: (50%)]	Loss: 0.127218
2020-09-21 02:39:40,118 DEBUG <timed exec>(l:112) - Train Worker charlie: (0%)]	Loss: 0.015373
2020-09-21 02:40:10,653 DEBUG <timed exec>(l:112) - Train Worker charlie: (50%)]	Loss: 0.031792
2020-09-21 02:41:04,099 DEBUG <timed exec>(l:112) - Train Worker bob: (0%)]	Loss: 0.107799
2020-09-21 02:41:34,553 DEBUG <timed exec>(l:112) - Train Worker bob: (50%)]	Loss: 0.172169
2020-09-21 02:42:05,150 DEBUG <timed exec>(l:112) - Train Worker alice: (0%)]	Loss: 0.069026
2020-09-21 02:42:35,822 DEBUG <timed exec>(l:112) - Train Worker alice: (50%)]	Loss:


Test set: Average loss: 0.1626, Accuracy: 60305/64000 (94.23%)

[0.0, 88.696875, 90.4203125, 91.215625, 91.8671875, 92.121875, 92.8203125, 93.2171875, 92.90625, 94.13125, 94.3296875, 94.3140625, 94.478125, 94.425, 94.4984375, 94.8828125, 94.784375, 95.05625, 94.3578125, 95.01875, 94.2265625]
Early stopping!
              precision    recall  f1-score   support

      benign       0.91      0.98      0.94     32052
         dga       0.98      0.90      0.94     31948

    accuracy                           0.94     64000
   macro avg       0.95      0.94      0.94     64000
weighted avg       0.94      0.94      0.94     64000



2020-09-21 03:26:15,877 DEBUG pyplot.py(l:225) - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


       0      1
0  31460    592
1   3103  28845
CPU times: user 5h 41min 14s, sys: 26min 14s, total: 6h 7min 28s
Wall time: 6h 8min 11s
